In [1]:
import pandas as pd
import requests
import os 

In [11]:
import requests
import os

def download_csvs(urls, folder_path):
    """
    Downloads a set of csv's from a list of urls and saves them in a folder.

    Parameters:
    urls (list): A list of urls to download csv's from.
    folder_path (str): The path of the folder to save the csv's in.

    Returns:
    None
    """
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    for url in urls:
        response = requests.get(url)
        file_name = url.split('/')[-1]
        file_path = os.path.join(folder_path, file_name)

        with open(file_path, 'wb') as f:
            f.write(response.content)
            print(f"{file_name} downloaded successfully!")


In [12]:
urls = [
    'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_2023_Jan_Jun.csv',
    'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_2022_Jul_Dec.csv',
    'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_2022_Jan_Jun.csv',
    'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_2021_Jul_Dec.csv',
    'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_2021_Jan_Jun.csv',
    'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_2020_Jul_Dec.csv',
    'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_2020_Jan_Jun.csv',
    'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_2019_Jul_Dec.csv',
    'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_2019_Jan_Jun.csv',
    'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_2018_Jul_Dec.csv',
    'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_2018_Jan_Jun.csv',
]

folder_path = '/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/resources/eia/6monthFiles'
download_csvs(urls, folder_path)

EIA930_BALANCE_2023_Jan_Jun.csv downloaded successfully!
EIA930_BALANCE_2022_Jul_Dec.csv downloaded successfully!
EIA930_BALANCE_2022_Jan_Jun.csv downloaded successfully!
EIA930_BALANCE_2021_Jul_Dec.csv downloaded successfully!
EIA930_BALANCE_2021_Jan_Jun.csv downloaded successfully!
EIA930_BALANCE_2020_Jul_Dec.csv downloaded successfully!
EIA930_BALANCE_2020_Jan_Jun.csv downloaded successfully!
EIA930_BALANCE_2019_Jul_Dec.csv downloaded successfully!
EIA930_BALANCE_2019_Jan_Jun.csv downloaded successfully!
EIA930_BALANCE_2018_Jul_Dec.csv downloaded successfully!
EIA930_BALANCE_2018_Jan_Jun.csv downloaded successfully!


### Process data

In [51]:
def read_and_concat_csvs(folder_path, columns_to_keep, output_folder_path):
    """
    Reads a set of csv's from a folder, removes all but a few specified columns, and concatenates them together.

    Parameters:
    folder_path (str): The path of the folder to read the csv's from.
    columns_to_keep (list): A list of column names to keep in the concatenated dataframe.
    output_folder_path (str): The path of the folder to save the concatenated csv.

    Returns:
    None
    """
    dfs = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            year = file_name.split('_')[2]
            if year not in dfs:
                dfs[year] = []
            df = pd.read_csv(file_path, usecols=columns_to_keep)
            df.columns = ['region','timestamp','demand_mw']
            dfs[year].append(df)

    for year, dfs_list in dfs.items():
        concatenated_df = pd.concat(dfs_list, ignore_index=True)

        concatenated_df = concatenated_df.set_index(['timestamp','region']).unstack(level=1)['demand_mw']
        concatenated_df.dropna(axis=1, how='all', inplace=True)

        output_file = os.path.join(output_folder_path, f'EIA_DMD_{year}.csv')
        concatenated_df.to_csv(output_file)
        print(f"{output_file} saved successfully!")


output_folder_path = '/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/resources/eia/processed'
columns_to_keep = ['UTC Time at End of Hour', 'Balancing Authority', 'Demand (MW)']
read_and_concat_csvs(folder_path, columns_to_keep, output_folder_path)

/var/folders/00/vqry7y9s78q3rvf_m_jmk_bc0000gn/T/ipykernel_98022/180512133.py:20: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=columns_to_keep)
/var/folders/00/vqry7y9s78q3rvf_m_jmk_bc0000gn/T/ipykernel_98022/180512133.py:20: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=columns_to_keep)
/var/folders/00/vqry7y9s78q3rvf_m_jmk_bc0000gn/T/ipykernel_98022/180512133.py:20: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=columns_to_keep)
/var/folders/00/vqry7y9s78q3rvf_m_jmk_bc0000gn/T/ipykernel_98022/180512133.py:20: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=columns_to_keep)
/var/folders/00/vqry7y9s78q3rvf_m_jmk_bc0000gn/T/ipy

/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/resources/eia/processed/EIA_DMD_2018.csv saved successfully!
/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/resources/eia/processed/EIA_DMD_2023.csv saved successfully!
/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/resources/eia/processed/EIA_DMD_2021.csv saved successfully!
/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/resources/eia/processed/EIA_DMD_2019.csv saved successfully!
/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/resources/eia/processed/EIA_DMD_2022.csv saved successfully!
/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/resources/eia/processed/EIA_DMD_2020.csv saved successfully!


In [16]:
df = pd.read_csv('/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/resources/eia/EIA_DMD_2022.csv')
# df.to_csv('/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/resources/eia/processed/EIA_DMD_2023.csv')

# df.set_index(['timestamp','region']).unstack(level=1)['demand_mw']
df.set_index('timestamp', inplace=True)
df.index = pd.to_datetime(df.index)


In [20]:
df.loc['2022-03-13']


,AECI,AVA,AZPS,BANC,BPAT,CHPD,CISO,CPLE,CPLW,DOPD,...,TAL,TEC,TEPC,TIDC,TPWR,TVA,WACM,WALC,WAUW,day
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-03-13 10:00:00,3244.0,1136.0,2573.0,1433.0,5580.0,201.0,19748.0,7283.0,901.0,215.0,...,324.0,1892.0,1326.0,229.0,500.0,23727.0,2231.0,1001.0,90.0,72
2022-03-13 22:00:00,2285.0,1491.0,2089.0,1483.0,6463.0,175.0,15670.0,7588.0,588.0,189.0,...,257.0,1975.0,1184.0,229.0,588.0,16678.0,2201.0,945.0,95.0,72
2022-03-13 11:00:00,3293.0,1188.0,2588.0,1396.0,5608.0,208.0,19275.0,7893.0,923.0,216.0,...,343.0,2019.0,1319.0,229.0,488.0,24151.0,2241.0,1012.0,89.0,72
2022-03-13 23:00:00,2344.0,1484.0,2279.0,1503.0,6375.0,170.0,16138.0,8198.0,609.0,186.0,...,260.0,2004.0,1222.0,237.0,584.0,16702.0,2225.0,936.0,97.0,72
2022-03-13 00:00:00,3256.0,1302.0,2422.0,1606.0,6174.0,193.0,18595.0,8503.0,921.0,201.0,...,292.0,2088.0,1345.0,254.0,562.0,22138.0,2332.0,982.0,100.0,72
2022-03-13 12:00:00,3316.0,1208.0,2661.0,1408.0,5721.0,213.0,19108.0,8725.5,961.0,221.0,...,363.0,2192.0,1362.0,227.0,502.0,24695.0,2235.0,1019.0,87.0,72
2022-03-13 01:00:00,3479.0,1328.0,2731.0,1644.0,6441.0,202.0,20554.0,8948.0,961.0,211.0,...,316.0,2144.0,1448.0,267.0,595.0,23578.0,2431.0,834.0,103.0,72
2022-03-13 13:00:00,3377.0,1241.0,2769.0,1420.0,5932.0,225.0,19196.0,9027.0,977.0,233.0,...,382.0,2392.0,1417.0,233.0,519.0,24990.0,2249.0,1044.0,89.0,72
2022-03-13 02:00:00,3625.0,1373.0,2990.0,1734.0,6746.0,211.0,22910.0,9106.0,947.0,218.0,...,320.0,2092.0,1492.0,273.0,607.0,23818.0,2547.0,863.0,99.0,72
